In [1]:
from keras.layers import Input, Dense, Dropout, Conv1D, Flatten, MaxPooling1D, Concatenate, Multiply, BatchNormalization
from keras.models import Model, Sequential
from sklearn.preprocessing import LabelEncoder
from keras import regularizers
from sklearn.model_selection import KFold 
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from keras.callbacks import Callback
from sklearn.linear_model import LogisticRegression, BayesianRidge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
import lightgbm as lgb
import math
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats
import pandas as pd 
import numpy as np
from sklearn.metrics import roc_curve, auc
import seaborn as sns
sns.set(style="whitegrid")
import matplotlib
from sklearn.naive_bayes import GaussianNB
np.random.seed(203)
import math
from scipy.stats import ks_2samp
from sklearn.ensemble import RandomForestRegressor
from keras.callbacks import ModelCheckpoint
import os


# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


Please consider upvoting this kernel if you find it helpful in any way. And don't hesitate to ask any questions on why I'm doing what I'm doing. And finally (most importantly), please point out if I'm being dumb anywhere below so that I can fix it :).

In [2]:
targets = pd.read_csv("y_train.csv")
df_train = pd.read_csv("X_train.csv")
df_test = pd.read_csv("X_test.csv")

Initially, I wanted to see how much predictive power the series had without any feature engineering. My plan on doing this was to create a Neural Net with multiple inputs (One for each 128 observation feature). With quite a lot of parameter tuning and regularization, the model was able to consistently achieve a CV accuracy between 61 and 67.

I then realized that to the model, certain textures were very similar (it kept on confusing the same classes). I wanted more features to be able to distinguish between these classes with a higher accuracy. Therefore, my goal was to create new features that aimed to establish the "smoothness" of all the readings. My hypthesis was that the smoothness of readings would help distinguish between similar classes. What are the "smoothness" features though? Firstly, the standard deviation of the differences between consecutive readings (a perfectly straight line will have a standard deviation of 0 because the differences are constant), and secondly, for more polynomial looking lines, the RMSE from the moving average of the line (If a set of readings has a lot of spikes, ie. isn't smooth, then the error wil be large).

Now, moving on to the actual code. In this first section, I'm just extracting all the values from the pandas dataframe and reshaping the resulting numpy array into its individual series (Plus rescaling).

In [3]:
targets = pd.get_dummies(targets, columns=["surface"])
Y = targets.drop(["series_id", "group_id"], axis=1).values
X = df_train.drop(["row_id", "series_id", "measurement_number"], axis=1).values
test = df_test.drop(["row_id", "series_id", "measurement_number"], axis=1).values
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.reshape((3810, 128, 10, 1))
X = X.transpose(0, 2, 1, 3)
test = scaler.transform(test)
test = test.reshape((3816, 128, 10, 1))
test = test.transpose(0, 2, 1, 3)

In the following code block, I'm using some pandas tricks to obtain the aforementioned "smoothness" features from the test and train dataframes.

In [4]:
#Every column gets the smoothness features, except for the ones in the list below
for col in df_train.drop(["row_id", "series_id", "measurement_number"], axis=1).columns:
    df_train["diff_"+col] = df_train[col]-df_train[col].shift(1)
    df_test["diff_"+col] = df_test[col]-df_test[col].shift(1)
    df_train["ma_"+col] = np.square(abs(df_train[col]-df_train[col].rolling(8).mean()))
    df_test["ma_"+col] = np.square(abs(df_test[col]-df_test[col].rolling(8).mean()))

# The first 8 measurement (0-7) contain data from the previous series id (because of the rolling mean), 
# so they have to be dropped
df_train = df_train[df_train["measurement_number"]>7]
df_test = df_test[df_test["measurement_number"]>7]

aggs = {
    "diff_orientation_X":"std",
    "diff_orientation_Y":"std",
    "diff_orientation_Z":"std",
    "diff_orientation_W":"std",
    "diff_angular_velocity_X":"std",
    "diff_angular_velocity_Y":"std",
    "diff_angular_velocity_Z":"std",
    "diff_linear_acceleration_X":"std",
    "diff_linear_acceleration_Y":"std",
    "diff_linear_acceleration_Z":"std",
    "ma_orientation_X":"sum",
    "ma_orientation_Y":"sum",
    "ma_orientation_Z":"sum",
    "ma_orientation_W":"sum",
    "ma_angular_velocity_X":"sum",
    "ma_angular_velocity_Y":"sum",
    "ma_angular_velocity_Z":"sum",
    "ma_linear_acceleration_X":"sum",
    "ma_linear_acceleration_Y":"sum",
    "ma_linear_acceleration_Z":"sum"
}

train_smooth = df_train.groupby("series_id").agg(aggs)
test_smooth = df_test.groupby("series_id").agg(aggs)
train_smooth.reset_index(inplace=True)
test_smooth.reset_index(inplace=True)

Just a quick check to ensure that the features turned out okay.

In [5]:
print(train_smooth.shape) #should be (3810, 20), because 10 columns got 2 additional features each
train_smooth.head()

(3810, 21)


,series_id,diff_orientation_X,diff_orientation_Y,diff_orientation_Z,diff_orientation_W,diff_angular_velocity_X,diff_angular_velocity_Y,diff_angular_velocity_Z,diff_linear_acceleration_X,diff_linear_acceleration_Y,...,ma_orientation_X,ma_orientation_Y,ma_orientation_Z,ma_orientation_W,ma_angular_velocity_X,ma_angular_velocity_Y,ma_angular_velocity_Z,ma_linear_acceleration_X,ma_linear_acceleration_Y,ma_linear_acceleration_Z
0,0,0.000019,0.000020,0.000064,0.000053,0.038691,0.028768,0.011237,0.922267,0.670404,...,3.812828e-07,4.730125e-07,0.000003,0.000001,0.157107,0.066516,0.015078,47.416411,54.790411,129.126698
1,1,0.000029,0.000083,0.000065,0.000130,0.076469,0.057924,0.025305,1.545196,1.353271,...,5.527609e-07,6.335519e-06,0.000002,0.000006,0.528547,0.249689,0.363267,113.848545,138.833410,450.217492
2,2,0.000031,0.000018,0.000069,0.000056,0.033881,0.044900,0.016317,1.132570,0.979919,...,2.602950e-06,8.248469e-07,0.000004,0.000002,0.164368,0.140227,0.035255,53.033779,144.875970,114.053852
3,3,0.000032,0.000046,0.000079,0.000236,0.126241,0.054900,0.022548,2.060239,2.278828,...,4.103109e-07,2.022280e-06,0.000003,0.000029,1.441615,0.223097,0.059015,199.561159,556.261437,934.213309
4,4,0.000022,0.000043,0.000028,0.000057,0.028686,0.012247,0.008092,0.256040,0.370395,...,9.975125e-06,4.312642e-05,0.000002,0.000002,0.091879,0.017231,0.056375,11.054523,32.276311,66.197295


In [6]:
X_smooth = train_smooth.drop(["series_id"], axis=1).values
test_smooth = test_smooth.drop(["series_id"], axis=1).values

scaler = StandardScaler()
X_smooth = scaler.fit_transform(X_smooth)
test_smooth = scaler.transform(test_smooth)

Now, to the fun part, building and training a Convolutional Neural Network and reformatting the input so that the CNN can train on it.

In [7]:
def init_model():
    
    #These features occur frequently throughout, so for easu of use, it's easier to change them up here.
    FIRST = 30 #20
    SECOND = 20 #10
    HEIGHT1 = 4 #4
    HEIGHT2 = 3 #4
    DROPOUT = 0.5
    STRIDES = None
    PS = 5
    
    input1 = Input(shape=(128, 1))
    a = Conv1D(FIRST, HEIGHT1, activation="relu", kernel_initializer="uniform")(input1)
    a = BatchNormalization()(a)
    a = MaxPooling1D(strides=STRIDES, pool_size=PS)(a)
    a = Conv1D(SECOND, HEIGHT2, activation="relu", kernel_initializer="uniform")(a)
    a = BatchNormalization()(a)
    a = MaxPooling1D(strides=STRIDES, pool_size=PS)(a)
    a = Flatten()(a)
    a = Dropout(DROPOUT)(a)

    input2 = Input(shape=(128, 1))
    b = Conv1D(FIRST, HEIGHT1, activation="relu", kernel_initializer="uniform")(input2)
    b = BatchNormalization()(b)
    b = MaxPooling1D(strides=STRIDES, pool_size=PS)(b)
    b = Conv1D(SECOND, HEIGHT2, activation="relu", kernel_initializer="uniform")(b)
    b = BatchNormalization()(b)
    b = MaxPooling1D(strides=STRIDES, pool_size=PS)(b)
    b = Flatten()(b)
    b = Dropout(DROPOUT)(b)

    input3 = Input(shape=(128, 1))
    c = Conv1D(FIRST, HEIGHT1, activation="relu", kernel_initializer="uniform")(input3)
    c = BatchNormalization()(c)
    c = MaxPooling1D(strides=STRIDES, pool_size=PS)(c)
    c = Conv1D(SECOND, HEIGHT2, activation="relu", kernel_initializer="uniform")(c)
    c = BatchNormalization()(c)
    c = MaxPooling1D(strides=STRIDES, pool_size=PS)(c)
    c = Flatten()(c)
    c = Dropout(DROPOUT)(c)

    input4 = Input(shape=(128, 1))
    d = Conv1D(FIRST, HEIGHT1, activation="relu", kernel_initializer="uniform")(input4)
    d = BatchNormalization()(d)
    d = MaxPooling1D(strides=STRIDES, pool_size=PS)(d)
    d = Conv1D(SECOND, HEIGHT2, activation="relu", kernel_initializer="uniform")(d)
    d = BatchNormalization()(d)
    d = MaxPooling1D(strides=STRIDES, pool_size=PS)(d)
    d = Flatten()(d)
    d = Dropout(DROPOUT)(d)

    input5 = Input(shape=(128, 1))
    e = Conv1D(FIRST, HEIGHT1, activation="relu", kernel_initializer="uniform")(input5)
    e = BatchNormalization()(e)
    e = MaxPooling1D(strides=STRIDES, pool_size=PS)(e)
    e = Conv1D(SECOND, HEIGHT2, activation="relu", kernel_initializer="uniform")(e)
    e = BatchNormalization()(e)
    e = MaxPooling1D(strides=STRIDES, pool_size=PS)(e)
    e = Flatten()(e)
    e = Dropout(DROPOUT)(e)

    input6 = Input(shape=(128, 1))
    f = Conv1D(FIRST, HEIGHT1, activation="relu", kernel_initializer="uniform")(input6)
    f = BatchNormalization()(f)
    f = MaxPooling1D(strides=STRIDES, pool_size=PS)(f)
    f = Conv1D(SECOND, HEIGHT2, activation="relu", kernel_initializer="uniform")(f)
    f = BatchNormalization()(f)
    f = MaxPooling1D(strides=STRIDES, pool_size=PS)(f)
    f = Flatten()(f)
    f = Dropout(DROPOUT)(f)

    input7 = Input(shape=(128, 1))
    g = Conv1D(FIRST, HEIGHT1, activation="relu", kernel_initializer="uniform")(input7)
    g = BatchNormalization()(g)
    g = MaxPooling1D(strides=STRIDES, pool_size=PS)(g)
    g = Conv1D(SECOND, HEIGHT2, activation="relu", kernel_initializer="uniform")(g)
    g = BatchNormalization()(g)
    g = MaxPooling1D(strides=STRIDES, pool_size=PS)(g)
    g = Flatten()(g)
    g = Dropout(DROPOUT)(g)

    input8 = Input(shape=(128, 1))
    h = Conv1D(FIRST, HEIGHT1, activation="relu", kernel_initializer="uniform")(input8)
    h = BatchNormalization()(h)
    h = MaxPooling1D(strides=STRIDES, pool_size=PS)(h)
    h = Conv1D(SECOND, HEIGHT2, activation="relu", kernel_initializer="uniform")(h)
    h = BatchNormalization()(h)
    h = MaxPooling1D(strides=STRIDES, pool_size=PS)(h)
    h = Flatten()(h)
    h = Dropout(DROPOUT)(h)

    input9 = Input(shape=(128, 1))
    i = Conv1D(FIRST, HEIGHT1, activation="relu", kernel_initializer="uniform")(input9)
    i = BatchNormalization()(i)
    i = MaxPooling1D(strides=STRIDES, pool_size=PS)(i)
    i = Conv1D(SECOND, HEIGHT2, activation="relu", kernel_initializer="uniform")(i)
    i = BatchNormalization()(i)
    i = MaxPooling1D(strides=STRIDES, pool_size=PS)(i)
    i = Flatten()(i)
    i = Dropout(DROPOUT)(i)

    input10 = Input(shape=(128, 1))
    j = Conv1D(FIRST, HEIGHT1, activation="relu", kernel_initializer="uniform")(input10)
    j = BatchNormalization()(j)
    j = MaxPooling1D(strides=STRIDES, pool_size=PS)(j)
    j = Conv1D(SECOND, HEIGHT2, activation="relu", kernel_initializer="uniform")(j)
    j = BatchNormalization()(j)
    j = MaxPooling1D(strides=STRIDES, pool_size=PS)(j)
    j = Flatten()(j)
    j = Dropout(DROPOUT)(j)
    
    input11 = Input(shape=(20,))
    k = Dense(30, activation="relu", kernel_initializer="uniform")(input11)
    k = Dropout(0.25)(k)

    merged = Concatenate()([a, b])
    merged = Concatenate()([merged, c])
    merged = Concatenate()([merged, d])
    merged = Concatenate()([merged, e])
    merged = Concatenate()([merged, f])
    merged = Concatenate()([merged, g])
    merged = Concatenate()([merged, h])
    merged = Concatenate()([merged, i])
    merged = Concatenate()([merged, j])
    merged = Concatenate()([merged, k])
    merged = Dense(30, activation = "relu", kernel_initializer="uniform")(merged)
    merged = Dropout(0.25)(merged)
    
    output = Dense(9, activation="softmax", kernel_initializer="uniform")(merged)
    model = Model([input1, input2, input3, input4, input5, input6, input7, input8, input9, input10, input11], output)
    return model

It turns out the LB score is almost always better with a higher number of folds. (Be warned, this might take a few hours to train).

In [8]:
te = [test[:,i] for i in range(10)]
te.append(test_smooth)

folds = StratifiedKFold(n_splits=15, shuffle=True, random_state=0)
oof = np.zeros((len(X), 9))
test_predictions = np.zeros((len(test), 9))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X,np.argmax(Y, axis=1))):
    print(fold_)
    X_train, X_test = X[trn_idx], X[val_idx]
    y_train, y_test = Y[trn_idx], Y[val_idx]
    X_smooth_train, X_smooth_test = X_smooth[trn_idx], X_smooth[val_idx]
    
    #The model needs a list of inputs. The code beneath this creates a column-input for each unique column in the training set and appends it to the list. 
    X_tr = [X_train[:,i] for i in range(10)]
    X_tr.append(X_smooth_train)
    X_te = [X_test[:,i] for i in range(10)]
    X_te.append(X_smooth_test)
    
    model = init_model()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    model.fit(X_tr, y_train, validation_data = (X_te, y_test), epochs=50, shuffle=True, class_weight="balanced")
    pre = model.predict(X_te)
    oof[val_idx] = model.predict(X_te)
    test_predictions+= model.predict(te)/20

0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 3552 samples, validate on 258 samples
Epoch 1/50
3552/3552 [==============================] - 14s 4ms/step - loss: 1.6594 - acc: 0.4091 - val_loss: 1.3364 - val_acc: 0.5039
Epoch 2/50
3552/3552 [==============================] - 3s 891us/step - loss: 1.3268 - acc: 0.5287 - val_loss: 1.1115 - val_acc: 0.6240
Epoch 3/50
3552/3552 [==============================] - 3s 882us/step - loss: 1.2084 - acc: 0.5681 - val_loss: 1.0887 - val_acc: 0.6124
Epoch 4/50
3552/3552 [==============================] - 3s 884us/step - loss: 1.0958 - acc: 0.6101 - val_loss: 0.9390 - val_acc: 0.6667
Epoch 5/50
3552/3552 [==============================] - 3s 895us/step - loss: 1.0378 - acc: 0.6292 - val_loss: 0.9330 - val_acc: 0.6395
Epoch 6/50
3552/3552 [===============

3552/3552 [==============================] - 3s 911us/step - loss: 1.1628 - acc: 0.5904 - val_loss: 1.0725 - val_acc: 0.6473
Epoch 4/50
3552/3552 [==============================] - 3s 925us/step - loss: 1.0683 - acc: 0.6287 - val_loss: 1.0129 - val_acc: 0.6550
Epoch 5/50
3552/3552 [==============================] - 3s 914us/step - loss: 1.0056 - acc: 0.6503 - val_loss: 0.9314 - val_acc: 0.6938
Epoch 6/50
3552/3552 [==============================] - 3s 909us/step - loss: 0.9560 - acc: 0.6593 - val_loss: 0.9373 - val_acc: 0.6783
Epoch 7/50
3552/3552 [==============================] - 3s 895us/step - loss: 0.9317 - acc: 0.6658 - val_loss: 0.8230 - val_acc: 0.7248
Epoch 8/50
3552/3552 [==============================] - 3s 906us/step - loss: 0.9028 - acc: 0.6805 - val_loss: 0.8417 - val_acc: 0.6938
Epoch 9/50
3552/3552 [==============================] - 3s 896us/step - loss: 0.8639 - acc: 0.7016 - val_loss: 0.7494 - val_acc: 0.7326
Epoch 10/50
3552/3552 [==============================] - 3s

3552/3552 [==============================] - 3s 944us/step - loss: 0.7712 - acc: 0.7227 - val_loss: 0.5796 - val_acc: 0.8023
Epoch 18/50
3552/3552 [==============================] - 3s 939us/step - loss: 0.7413 - acc: 0.7385 - val_loss: 0.6293 - val_acc: 0.8023
Epoch 19/50
3552/3552 [==============================] - 3s 927us/step - loss: 0.7408 - acc: 0.7393 - val_loss: 0.5809 - val_acc: 0.7907
Epoch 20/50
3552/3552 [==============================] - 3s 916us/step - loss: 0.7157 - acc: 0.7421 - val_loss: 0.5889 - val_acc: 0.8023
Epoch 21/50
3552/3552 [==============================] - 3s 925us/step - loss: 0.7136 - acc: 0.7458 - val_loss: 0.5889 - val_acc: 0.8295
Epoch 22/50
3552/3552 [==============================] - 3s 924us/step - loss: 0.7024 - acc: 0.7531 - val_loss: 0.5688 - val_acc: 0.8101
Epoch 23/50
3552/3552 [==============================] - 3s 924us/step - loss: 0.7141 - acc: 0.7447 - val_loss: 0.5278 - val_acc: 0.8256
Epoch 24/50
3552/3552 [==============================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



3553/3553 [==============================] - 3s 942us/step - loss: 0.6048 - acc: 0.7703 - val_loss: 0.6174 - val_acc: 0.7821
Epoch 47/50
3553/3553 [==============================] - 3s 977us/step - loss: 0.6031 - acc: 0.7740 - val_loss: 0.5041 - val_acc: 0.8132
Epoch 48/50
3553/3553 [==============================] - 3s 964us/step - loss: 0.6207 - acc: 0.7684 - val_loss: 0.5019 - val_acc: 0.8210
Epoch 49/50
3553/3553 [==============================] - 3s 974us/step - loss: 0.5957 - acc: 0.7715 - val_loss: 0.5142 - val_acc: 0.8210
Epoch 50/50
3553/3553 [==============================] - 3s 959us/step - loss: 0.6269 - acc: 0.7701 - val_loss: 0.4850 - val_acc: 0.8249
4
Train on 3554 samples, validate on 256 samples
Epoch 1/50
3554/3554 [==============================] - 12s 3ms/step - loss: 1.6517 - acc: 0.4218 - val_loss: 1.3882 - val_acc: 0.5898
Epoch 2/50
3554/3554 [==============================] - 4s 989us/step - loss: 1.2779 - acc: 0.5667 - val_loss: 1.2670 - val_acc: 0.6133
Epoch 3

...

3558/3558 [==============================] - 4s 1ms/step - loss: 0.7068 - acc: 0.7411 - val_loss: 0.5168 - val_acc: 0.8016
Epoch 23/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.7307 - acc: 0.7400 - val_loss: 0.5349 - val_acc: 0.8056
Epoch 24/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.7384 - acc: 0.7375 - val_loss: 0.5390 - val_acc: 0.8056
Epoch 25/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.7086 - acc: 0.7434 - val_loss: 0.5205 - val_acc: 0.8294
Epoch 26/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.6894 - acc: 0.7437 - val_loss: 0.5359 - val_acc: 0.8095
Epoch 27/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.6794 - acc: 0.7493 - val_loss: 0.5093 - val_acc: 0.8095
Epoch 28/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.6839 - acc: 0.7485 - val_loss: 0.4917 - val_acc: 0.8294
Epoch 29/50
3558/3558 [==============================] - 4s 1ms/ste

3558/3558 [==============================] - 4s 1ms/step - loss: 0.6410 - acc: 0.7552 - val_loss: 0.5460 - val_acc: 0.8095
Epoch 33/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.6503 - acc: 0.7617 - val_loss: 0.5227 - val_acc: 0.8016
Epoch 34/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.6476 - acc: 0.7583 - val_loss: 0.5549 - val_acc: 0.7857
Epoch 35/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.6134 - acc: 0.7743 - val_loss: 0.5577 - val_acc: 0.7738
Epoch 36/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.6472 - acc: 0.7622 - val_loss: 0.5732 - val_acc: 0.7976
Epoch 37/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.6222 - acc: 0.7718 - val_loss: 0.5962 - val_acc: 0.7778
Epoch 38/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.6160 - acc: 0.7693 - val_loss: 0.5328 - val_acc: 0.8016
Epoch 39/50
3558/3558 [==============================] - 4s 1ms/ste

3558/3558 [==============================] - 4s 1ms/step - loss: 0.6287 - acc: 0.7749 - val_loss: 0.4419 - val_acc: 0.8373
Epoch 43/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.6289 - acc: 0.7721 - val_loss: 0.4361 - val_acc: 0.8690
Epoch 44/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.6172 - acc: 0.7749 - val_loss: 0.4646 - val_acc: 0.8690
Epoch 45/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.6125 - acc: 0.7788 - val_loss: 0.5101 - val_acc: 0.8095
Epoch 46/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.5824 - acc: 0.7892 - val_loss: 0.4347 - val_acc: 0.8294
Epoch 47/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.5891 - acc: 0.7782 - val_loss: 0.4589 - val_acc: 0.8373
Epoch 48/50
3558/3558 [==============================] - 4s 1ms/step - loss: 0.6207 - acc: 0.7791 - val_loss: 0.4712 - val_acc: 0.8333
Epoch 49/50
3558/3558 [==============================] - 4s 1ms/ste

3560/3560 [==============================] - 4s 1ms/step - loss: 1.2962 - acc: 0.5458 - val_loss: 1.2060 - val_acc: 0.5480
Epoch 3/50
3560/3560 [==============================] - 4s 1ms/step - loss: 1.1829 - acc: 0.5893 - val_loss: 1.0010 - val_acc: 0.6400
Epoch 4/50
3560/3560 [==============================] - 4s 1ms/step - loss: 1.0978 - acc: 0.6070 - val_loss: 0.9402 - val_acc: 0.6840
Epoch 5/50
3560/3560 [==============================] - 4s 1ms/step - loss: 1.0452 - acc: 0.6309 - val_loss: 0.9071 - val_acc: 0.6720
Epoch 6/50
3560/3560 [==============================] - 4s 1ms/step - loss: 0.9833 - acc: 0.6514 - val_loss: 0.7855 - val_acc: 0.7240
Epoch 7/50
3560/3560 [==============================] - 4s 1ms/step - loss: 0.9658 - acc: 0.6632 - val_loss: 0.7830 - val_acc: 0.7280
Epoch 8/50
3560/3560 [==============================] - 4s 1ms/step - loss: 0.9291 - acc: 0.6674 - val_loss: 0.7118 - val_acc: 0.7600
Epoch 9/50
3560/3560 [==============================] - 4s 1ms/step - los

3561/3561 [==============================] - 4s 1ms/step - loss: 0.8117 - acc: 0.7102 - val_loss: 0.6939 - val_acc: 0.7671
Epoch 13/50
3561/3561 [==============================] - 4s 1ms/step - loss: 0.7819 - acc: 0.7155 - val_loss: 0.7402 - val_acc: 0.7349
Epoch 14/50
3561/3561 [==============================] - 4s 1ms/step - loss: 0.7672 - acc: 0.7231 - val_loss: 0.7918 - val_acc: 0.7470
Epoch 15/50
3561/3561 [==============================] - 4s 1ms/step - loss: 0.7711 - acc: 0.7206 - val_loss: 0.7325 - val_acc: 0.7590
Epoch 16/50
3561/3561 [==============================] - 4s 1ms/step - loss: 0.7551 - acc: 0.7256 - val_loss: 0.7123 - val_acc: 0.7590
Epoch 17/50
3561/3561 [==============================] - 4s 1ms/step - loss: 0.7456 - acc: 0.7408 - val_loss: 0.6838 - val_acc: 0.7550
Epoch 18/50
3561/3561 [==============================] - 4s 1ms/step - loss: 0.7682 - acc: 0.7209 - val_loss: 0.7118 - val_acc: 0.7470
Epoch 19/50
3561/3561 [==============================] - 4s 1ms/ste

In [9]:
print(test_predictions)

[[1.75828655e-06 9.10173607e-03 1.33672978e-01 ... 3.23036511e-10
  2.00083961e-01 3.27775574e-04]
 [5.42897530e-01 1.60125344e-01 1.16976081e-02 ... 3.28308030e-03
  2.50941830e-03 1.25488497e-02]
 [2.31603237e-06 6.33470642e-03 1.49466586e-01 ... 6.83435571e-10
  5.93096824e-01 9.37274473e-04]
 ...
 [1.27595310e-01 1.87077186e-01 1.67241357e-01 ... 5.37498143e-04
  7.41873180e-03 1.00979551e-01]
 [1.47051250e-01 4.03664744e-01 1.11552446e-03 ... 9.64029145e-06
  3.00440656e-02 1.67196553e-01]
 [4.19670059e-02 5.51529306e-03 2.10127875e-03 ... 3.55652082e-06
  3.34031521e-03 6.94200329e-01]]


The class probability predictions need to be converted to a surface text prediction. The code below does exactly that. 

In [10]:

predictions = np.argmax(test_predictions, axis=1)
true = np.argmax(y_test, axis=1)

labels = {
    0:"carpet",
    1:"concrete",
    2:"fine_concrete",
    3:"hard_tiles",
    4:"hard_tiles_large_space",
    5:"soft_pvc",
    6:"soft_tiles",
    7:"tiled",
    8:"wood"
}

pre = []
for p in predictions:
    pre.append(labels[p])

In [12]:
sub = pd.read_csv("sample_submission.csv")
sub["surface"] = np.array(pre)
sub.to_csv("submission.csv", index=False)